In [1]:
import faiss
from tqdm import tqdm
import numpy as np
import torch
from SimCSERetrieval import SimCSERetrieval

In [3]:
fname = "./data/all.txt"
pretrained = "hfl/chinese-bert-wwm-ext"
simcse_model = "./model/epoch_16-batch_6000-loss_0.029720"
batch_size = 64
max_length = 100
device = "cpu"

In [4]:
simcse = SimCSERetrieval(fname, pretrained, simcse_model, batch_size, max_length, device)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
simcse.encode_file()

22547it [00:00, 446946.54it/s]


In [6]:
simcse.index = faiss.read_index("./data/simcse.trained.index")
print(simcse.index.ntotal)

22547


In [ ]:
sentence = "福田外环车流大，车辆慢行"

vec = simcse.encode_batch([sentence])
vec = vec / vec.norm(dim=1, keepdim=True)
vec = vec.cpu().numpy()
_, sim_idx = simcse.index.search(vec, 20) #D, I
for i in range(sim_idx.shape[1]):
  idx = sim_idx[0, i]
  dis = 1 - _[0, i]
  if idx > -1 and dis > 0:
    print(idx, simcse.id2text[idx], dis)

In [7]:
sentence = "福田外环车流大，车辆慢行"
res = simcse.sim_query(sentence, 5)
print(res)

[{'text': '福田外环车流大，车辆慢行', 'similarity': 0.9999999999995794}, {'text': '福田出口外环车流大，车辆慢行', 'similarity': 0.6166631579399109}, {'text': '福田出口外环车流大，车辆慢行', 'similarity': 0.6166631579399109}, {'text': '福田出口外环车流大，车辆慢行。', 'similarity': 0.574103981256485}, {'text': '福永外环车流大，车辆慢行', 'similarity': 0.5403066575527191}]
